In [7]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [8]:
data = datasets.load_iris()

df = pd.DataFrame(data.data)
df.columns = data.feature_names
df['target'] = data.target
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [9]:
# Decision tree class having attributes as
# feature to split over on that node, the boundary for splitting & the children nodes

class DecisionTree:
    def __init__(self, f, f_val):
        self.f = f
        self.f_val = f_val
        self.children = []

In [10]:
def entropy(df):
    # making set of all target values
    target_val = set(df.target.values)
    p = []
    # finding no. of samples in each class
    for target in target_val:
        p.append(len(df[df.target == target]))
    p = np.array(p)
    # probability and log2(prob.) calculation for each class
    p = p/p.sum()
    log2_p = np.log10(p)/np.log10(2*np.ones(len(p)))
    # entropy/info gain calculation
    ent = (-p*log2_p).sum()
    return ent

def combined_entropy(df1, df2):
    ent1 = entropy(df1)
    ent2 = entropy(df2)
    l1 = len(df1)
    l2 = len(df2)
    # calculation of combined-weighted entropy
    c_ent = ((l1*ent1) + (l2*ent2))/(l1+l2)
    return c_ent

def gain_ratio(split_df):
    df1 = split_df[0]
    df2 = split_df[1]
    df = df1.append(df2)
    ent_i = entropy(df)
    ent_f = combined_entropy(df1, df2)
    splits_arr = np.array([len(df1), len(df2)])
    # splits' probabilty and log2(prob.)
    splits_arr = splits_arr/splits_arr.sum()
    log2_splits_arr = np.log10(splits_arr)/np.log10(2*np.ones(len(splits_arr)))
    # split info calculation
    split_info = (-splits_arr*log2_splits_arr).sum()
    # returning gain ratio
    return abs(ent_f - ent_i)/split_info

def split(df, f_name, f_val):
    # splitting dataframe given the feature name and boundary value
    df1 = df[df[f_name] <= f_val]
    df2 = df[df[f_name] > f_val]
    return df1, df2

In [11]:
def printDecisionTree(df, level = 0, visited_f = set(['target'])):
    
    unvisited_f = set(df.columns).difference(visited_f)
    target_set = set(df.target.values)
    ent = entropy(df)
    
    # checking edge conditions of recursion
    if len(target_set) == 1:
        print('LEVEL {}:'.format(level))
        print('Total Count = {}'.format(len(df)))
        for ele in target_set:
            print('Count of {} ({}) = {}'.format(ele, data.target_names[ele], len(df[df['target'] == ele])))
        print('Current Entropy = {}'.format(ent))
        target_class = target_set.pop()
        print('It is a PURE NODE, classified as: {} ({})'.format(target_class, data.target_names[target_class]))
        print()
        return target_class
    if len(unvisited_f) == 0:
        print('LEVEL {}:'.format(level))
        print('Total Count = {}'.format(len(df)))
        for ele in target_set:
            print('Count of {} ({}) = {}'.format(ele, data.target_names[ele], len(df[df['target'] == ele])))
        print('Current Entropy = {}'.format(ent))
        # finding majority class
        li = [0]*len(data.target_names)
        for ele in target_set:
            li[ele] = len(df[df['target'] == ele])
        target_class = li.index(max(li))
        print('NO MORE FEATURES left to split, classified as: {} ({})'.format(target_class, data.target_names[target_class]))
        print()
        return target_class
    
    # finding best boundary value in each feature
    d = {}
    for f in unvisited_f:
        max_f_val = 0
        max_gr = 0
        f_set = set(df[f].values)
        for f_val in f_set:
            gr = gain_ratio(split(df, f, f_val))
            if gr > max_gr:
                max_gr = gr
                max_f_val = f_val
        d[f] = (max_f_val, max_gr)
    # finding best feature to split upon
    best_f = max(d, key = lambda ele : d[ele][1])
    
    # printing statements
    print('LEVEL {}:'.format(level))
    print('Total Count = {}'.format(len(df)))
    for ele in target_set:
        print('Count of {} ({}) = {}'.format(ele, data.target_names[ele], len(df[df['target'] == ele])))
    print('Current Entropy = {}'.format(ent))
    print('Splitting on feature {} with boundary {} having gain ratio {}'.format(best_f, d[best_f][0], d[best_f][1]))
    print()
    
    # splitting the dataset
    df1, df2 = split(df, best_f, d[best_f][0])
    visited_f.add(best_f)
    
    # building the tree
    root = DecisionTree(best_f, d[best_f][0])
    child1 = printDecisionTree(df1, level+1, visited_f)
    root.children.append(child1)
    child2 = printDecisionTree(df2, level+1, visited_f)
    root.children.append(child2)
    return root

In [12]:
printDecisionTree(df)

<ipython-input-10-e2eccd64b557>:34: RuntimeWarning: divide by zero encountered in log10
  log2_splits_arr = np.log10(splits_arr)/np.log10(2*np.ones(len(splits_arr)))
<ipython-input-10-e2eccd64b557>:36: RuntimeWarning: invalid value encountered in multiply
  split_info = (-splits_arr*log2_splits_arr).sum()


LEVEL 0:
Total Count = 150
Count of 0 (setosa) = 50
Count of 1 (versicolor) = 50
Count of 2 (virginica) = 50
Current Entropy = 1.5849625007211559
Splitting on feature petal length (cm) with boundary 1.9 having gain ratio 0.9999999999999998

LEVEL 1:
Total Count = 50
Count of 0 (setosa) = 50
Current Entropy = 0.0
It is a PURE NODE, classified as: 0 (setosa)

LEVEL 1:
Total Count = 100
Count of 1 (versicolor) = 50
Count of 2 (virginica) = 50
Current Entropy = 1.0
Splitting on feature petal width (cm) with boundary 1.7 having gain ratio 0.6933647985912663

LEVEL 2:
Total Count = 54
Count of 1 (versicolor) = 49
Count of 2 (virginica) = 5
Current Entropy = 0.44506485705083854
Splitting on feature sepal length (cm) with boundary 7.0 having gain ratio 0.4975543406112364

LEVEL 3:
Total Count = 53
Count of 1 (versicolor) = 49
Count of 2 (virginica) = 4
Current Entropy = 0.3860189005698934
Splitting on feature sepal width (cm) with boundary 2.8 having gain ratio 0.06283947273220815

LEVEL 4:
To